In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from trl import SFTTrainer
import torch


In [ ]:

model_name = "deepseek-ai/deepseek-llm-7b-instruct"  # or any DeepSeek distilled variant
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)


In [ ]:

model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)


In [ ]:

dataset = load_dataset("json", data_files="your_dataset.json")["train"]

def formatting_func(example):
    return f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example.get('input','')}\n\n### Response:\n{example['output']}"


In [ ]:

args = TrainingArguments(
    output_dir="./deepseek-lora-checkpoints",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none",
    remove_unused_columns=False
)


In [ ]:

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    formatting_func=formatting_func,
    args=args,
    packing=False
)


In [ ]:

trainer.train()


In [ ]:
trainer.model.save_pretrained("deepseek-lora-adapter")
